In [1]:
import sys
sys.path.append("/u/dssc/zenocosini/helm_suite/MCQA_Benchmark")
from metrics.utils import hidden_states_collapse
from metrics.query import DataFrameQuery
from common.tensor_storage import TensorStorage
from common.metadata_db import MetadataDB
from metrics.utils import  exact_match, angular_distance

#from sklearn.feature_selection import mutual_info_regression MISSIN?
from dadapy.data import Data

from pathlib  import Path

import numpy as np
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer

from common.metadata_db import MetadataDB
from common.utils import *
from pathlib import Path
import pickle

In [2]:
def set_dataframes(db) -> pd.DataFrame:
    """
    Aggregate in a dataframe the hidden states of all instances
    ----------
    hidden_states: pd.DataFrame(num_instances, num_layers, model_dim)
    """
    df = pd.read_sql("SELECT * FROM metadata", db.conn)
    df["train_instances"] = df["train_instances"].astype(str)
    df.drop(columns=["id"],inplace = True)
    #import pdb; pdb.set_trace()
    df.drop_duplicates(subset = ["id_instance"],inplace = True, ignore_index = True) # why there are duplicates???
    return df

In [3]:
def tensor_retrieve(dict_query):
    query = DataFrameQuery(dict_query)
    hidden_states,logits, hidden_states_df= hidden_states_collapse(metadata_df,tensor_storage,query)
    return hidden_states,logits,hidden_states_df

In [4]:
def constructing_labels(label: str, hidden_states_df: pd.DataFrame, hidden_states: np.ndarray) -> np.ndarray:
    labels_literals = hidden_states_df[label].unique()
    labels_literals.sort()
    
    map_labels = {class_name: n for n,class_name in enumerate(labels_literals)}
    
    label_per_row = hidden_states_df[label].reset_index(drop=True)
    label_per_row = np.array([map_labels[class_name] for class_name in label_per_row])[:hidden_states.shape[0]]
    
    return label_per_row, map_labels

In [5]:
_PATH = Path("/orfeo/scratch/dssc/zenocosini/mmlu_result/")

metadata_db = MetadataDB(_PATH / "metadata.db")
metadata_df = set_dataframes(metadata_db)
tensor_storage = TensorStorage(Path(_PATH, "tensor_files"))

In [8]:
!ls /orfeo/scratch/dssc/zenocosini/mmlu_result/log_reg

accuracies.pkl			  model_layer18.joblib	model_layer2.joblib
cross_val_accuracies_letter.pkl   model_layer19.joblib	model_layer30.joblib
cross_val_accuracies.pkl	  model_layer1.joblib	model_layer31.joblib
cross_val_accuracies_subject.pkl  model_layer20.joblib	model_layer32.joblib
model_layer0.joblib		  model_layer21.joblib	model_layer3.joblib
model_layer10.joblib		  model_layer22.joblib	model_layer4.joblib
model_layer11.joblib		  model_layer23.joblib	model_layer5.joblib
model_layer12.joblib		  model_layer24.joblib	model_layer6.joblib
model_layer13.joblib		  model_layer25.joblib	model_layer7.joblib
model_layer14.joblib		  model_layer26.joblib	model_layer8.joblib
model_layer15.joblib		  model_layer27.joblib	model_layer9.joblib
model_layer16.joblib		  model_layer28.joblib	test_model.joblib
model_layer17.joblib		  model_layer29.joblib


In [6]:
df_linear_probe = pd.read_pickle(_PATH / "log_reg/cross_val_accuracies_subject.pkl")

In [7]:
df_linear_probe

,0,2,5
0,0.007135,0.007135,0.007135
1,0.473350,0.705458,0.733643
2,0.612772,0.938709,0.958188
3,0.788084,0.988441,0.996932
4,0.865929,0.998430,0.999429
5,0.951980,0.999929,0.999786
6,0.976597,0.999929,0.999929
7,0.989654,0.999929,1.000000
8,0.989226,0.999857,0.999929
9,0.990082,1.000000,0.999929
